In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = 'plotly_white'

import warnings
warnings.filterwarnings("ignore")

In [2]:
bdt_J19 = pd.read_csv('df_bdt_eval.csv')
bdt_J19_5M_23 = bdt_J19[(bdt_J19['opt']==23) & (bdt_J19['model']=='5M')]
bdt_J19_5M_0 = bdt_J19[(bdt_J19['opt']==0) & (bdt_J19['model']=='5M')]

In [3]:
vm = input()
if vm.lower()=='yes':
    path='/mnt/cephfs/ml_data/mc_2021/'
else:
    path=''

yes


In [4]:
import pickle
from tensorflow.keras.models import load_model

model_type = input('xgb or nn: ', )
models = []
if model_type=='xgb':
    models.append(pickle.load(open("models/16_features_max_depth_10/xgb_energy_ideal_opt_16.dat", "rb")))
    models.append(pickle.load(open("models/16_features_max_depth_10/xgb_energy_real_opt_16.dat", "rb")))
elif model_type=='nn':
    models.append(load_model("models/5M_ideal.h5"))
    models.append(load_model("models/5M_real.h5"))

xgb or nn: nn


In [5]:
# opt_features = ['AccumCharge', 'R_cht', 'z_cc', 'pe_std',
#                 'nPMTs', 'ht_kurtosis', 'ht_25-20p', 'R_cc',
#                 'ht_5-2p', 'pe_mean', 'jacob_cht', 'phi_cc',
#                 'ht_35-30p', 'ht_20-15p', 'pe_35p', 'ht_30-25p']

In [6]:
opt_features = ['AccumCharge', 'nPMTs', 'R_cc', 'R_cht', 'pe_mean',
                'pe_std', 'pe_skew', 'pe_kurtosis', 'pho_cc', 'pho_cht',
                'ht_2p', 'ht_5p', 'ht_10p', 'ht_15p', 'ht_20p',
                'ht_25p', 'ht_30p', 'ht_35p', 'ht_40p', 'ht_45p',
                'ht_50p', 'ht_55p', 'ht_60p', 'ht_65p', 'ht_70p',
                'ht_75p', 'ht_80p', 'ht_85p', 'ht_90p', 'ht_95p']

In [7]:
scalers = []
scalers.append(pickle.load(open('models/scaler_ideal.pkl','rb')))
scalers.append(pickle.load(open('models/scaler_real.pkl','rb')))

In [39]:
%%time
Rs = np.linspace(14, 17.2, 33)
energies = [0, 0.1, 0.3, 0.6] + list(range(1, 11))

y_true_all = []
y_pred_all = []
for j in tqdm(range(len(models)), "Models...", leave=False):
    y_true_array = []
    y_pred_array = []
    if j == 0:
        name = 'ProcessedTestIdeal'
    elif j == 1:
        name = 'ProcessedTestReal'
    for energy in tqdm(energies, "Energies...", leave=False):
        test = pd.read_csv('{}processed_data/{}/{}MeV.csv.gz'.format(path, name, energy))
        edep = np.array(test['edep'])
        X_test = test.iloc[:, :-5][opt_features]
        X_test = scalers[j].transform(X_test)
        edep_preds = models[j].predict(X_test).flatten()

        y_true = []
        y_pred = []
        for k in tqdm(range(len(Rs)), "Rs...", leave=False): 
            y_true.append(edep[test['edepR'] < Rs[k]])
            y_pred.append(edep_preds[test['edepR'] < Rs[k]])

        y_true_array.append(y_true)
        y_pred_array.append(y_pred)

    y_true_all.append(y_true_array)
    y_pred_all.append(y_pred_array)

Models...:   0%|          | 0/2 [00:00<?, ?it/s]

Energies...:   0%|          | 0/14 [00:00<?, ?it/s]

Rs...:   0%|          | 0/33 [00:00<?, ?it/s]

Rs...:   0%|          | 0/33 [00:00<?, ?it/s]

Rs...:   0%|          | 0/33 [00:00<?, ?it/s]

...

Rs...:   0%|          | 0/33 [00:00<?, ?it/s]

Rs...:   0%|          | 0/33 [00:00<?, ?it/s]

Rs...:   0%|          | 0/33 [00:00<?, ?it/s]

Rs...:   0%|          | 0/33 [00:00<?, ?it/s]

CPU times: user 11min 37s, sys: 1min 27s, total: 13min 4s
Wall time: 8min 6s


In [44]:
np.array(y_true_all).shape

(2, 14, 33)

In [56]:
diffs = np.array([
    [[y_pred_all[k][j][i] - y_true_all[k][j][i] for i in range(len(y_pred_array[0]))]
    for j in range(len(y_true_all[0]))]
    for k in range(len(y_true_all))
])

In [57]:
energies = [0, 0.1, 0.3, 0.6] + list(range(1, 11))
energies = np.array([1.022+i for i in energies]).round(5)
energies

array([ 1.022,  1.122,  1.322,  1.622,  2.022,  3.022,  4.022,  5.022,
        6.022,  7.022,  8.022,  9.022, 10.022, 11.022])

In [59]:
diffs.shape

(2, 14, 33)

In [60]:
[diffs[0][0][i].shape for i in range(len(Rs))]

[(49396,),
 (50474,),
 (51584,),
 (52647,),
 (53766,),
 (54898,),
 (56026,),
 (57174,),
 (58329,),
 (59597,),
 (60795,),
 (62097,),
 (63355,),
 (64562,),
 (65776,),
 (67081,),
 (68367,),
 (69648,),
 (71016,),
 (72361,),
 (73736,),
 (75115,),
 (76548,),
 (77968,),
 (79375,),
 (80783,),
 (82316,),
 (83822,),
 (85343,),
 (86924,),
 (88477,),
 (90127,),
 (91767,)]

In [131]:
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.stats import norm

a_all = []
errors_all = []
for i in tqdm(range(diffs.shape[0])):
    a_array = []
    errors_array = []
    for j in range(diffs.shape[1]):
        a = []
        e = []
        for k in range(diffs.shape[2]):
            fig, ax = plt.subplots()
            nbins = 150
            n, bins, patches = ax.hist(diffs[i][j][k], nbins, density=True, facecolor = 'grey', alpha = 0.5, label='before');
            plt.close(fig)
            centers = (0.5*(bins[1:]+bins[:-1]))
            pars, cov = curve_fit(lambda x, mu, sig : norm.pdf(x, loc=mu, scale=sig), centers, n, p0=[0,1])  
            a.append(pars)
            e.append(cov)
        a_array.append(a)
        errors_array.append(e)
    a_all.append(a_array)
    errors_all.append(errors_array)

  0%|          | 0/2 [00:00<?, ?it/s]

In [139]:
# from scipy import stats
# fig = go.Figure()

# N = -1

# for i in range(len(diffs[1])): 
#     x = np.linspace(diffs[1][i][N].min(), diffs[1][i][N].max(), 100)
#     p = stats.norm.pdf(x, a_all[1][i][N][0], a_all[1][i][N][1])

#     fig.add_trace(go.Scatter(x=x,
#                  y=p, mode='lines', name='mu={:.3f} +- {:.3f}, sigma={:.3f} +- {:.3f}'.format(
#                      a_all[1][i][N][0], np.sqrt(errors_all[1][i][N][0][0]),
#                      a_all[1][i][N][1], np.sqrt(errors_all[1][i][N][1][1])),
#                     visible = (i==0)
#                 ))

# for i in range(len(diffs[1])): 
#     fig.add_trace(go.Histogram(x=diffs[1][i][N], xbins=dict(size=0.005),
#                   showlegend=False, histnorm='probability density',
#                  visible = (i==0)
#                 ))

# buttons = []
# for N in range(len(diffs[1])): 
#     buttons.append(
#         dict(
#              args=['visible', [False]*N + [True] + [False]*(len(diffs[1])-1-N)],
#                  label='Energy =  {} MeV'.format(energies[N]),
#              method='restyle'
#         )
#     )

# fig.update_layout(

#     xaxis = dict(
#         showline=True,
#         ticks='outside',
#         mirror=True,
#         linecolor='black',
#         showgrid=True,
#         gridcolor='grey',
#         gridwidth=0.25,
#     ),

#     yaxis = dict(
#         showline=True,
#         ticks='outside',
#         mirror=True,
#         linecolor='black',
#         tick0=0,
# #             dtick=1,
#         showgrid=True,
#         gridcolor='grey',
#         gridwidth=0.25,
#         zeroline=True,
#         zerolinecolor='black',
#         zerolinewidth=0.25
#         ),
# )


# fig.update_layout(
#     xaxis_title=r"$$E_{rec} - E_{true}$$",
# #         yaxis_title="y",
#     showlegend=True,
#     updatemenus=list([
#         dict(
#             x=0.5,
#             y=1.2,
#             yanchor='top',
#             buttons=buttons
#         ),
#     ]),
#     legend=dict(
#         orientation="h",
#         yanchor="bottom",
#         y=1.05,
#         xanchor="right",
#         x=1
#     )
# )

# fig.show()

In [132]:
error_sigma_all = []
for ind in range(diffs.shape[0]):
    error_sigma = []
    for k in range(diffs.shape[2]):
        error = [100 * np.sqrt(errors_all[ind][i][k][1][1]) / energies[i] for i in range(1, len(energies)-1)]
        error_sigma.append(error)
    error_sigma_all.append(error_sigma)

error_mu_all = []
for ind in range(diffs.shape[0]):
    error_mu = []
    for k in range(diffs.shape[2]):
        error = [100 * np.sqrt(errors_all[ind][i][k][1][1]) / energies[i] for i in range(1, len(energies)-1)]
        error_mu.append(error)
    error_mu_all.append(error_mu)

In [86]:
res_all = []
bias_all = []
for ind in range(diffs.shape[0]):
    res = []
    bias = []
    for k in range(diffs.shape[2]):
        sigma = [100 * a_all[ind][i][k][1] / energies[i] for i in range(1, len(energies)-1)]
        mu = [100 * a_all[ind][i][k][0] / energies[i] for i in range(1, len(energies)-1)]
        res.append(sigma)
        bias.append(mu)
        
    res_all.append(res)
    bias_all.append(bias)

In [88]:
def a(x, a):
    return np.sqrt((a/x**0.5)**2)


def b(x, b):
    b_list = []
    b_list.append(np.sqrt(b**2))
    return b_list*len(x)


def c(x, c):
    return np.sqrt((c/x)**2)


def func(x, a, b, c):
    return np.sqrt((a/x**0.5)**2 + b**2 + (c/x)**2) 


def approximated(x, y, yerr):
    popt, pcov = curve_fit(func, x, y, sigma=yerr, maxfev=10**8, bounds=([0, 0, 0.25], [5, 5, 5]))
    a, b, c = popt
    #perr = np.sqrt(abs(pcov.diagonal()))

    return func(x, a, b, c), popt, pcov

In [90]:
fig = go.Figure()

fig.add_trace(
    go.Surface(
        x=energies[1:-1],
        y=Rs,
        z=np.array(res_all)[1, :, :],
        colorscale='inferno'
    )
)

fig.update_layout(
    autosize=True,
    title="Real",
    scene = dict(
        xaxis_title='Visible energy, MeV',
        yaxis_title='R, m',
        zaxis_title='Resolution, %',
    ),
    width=800,
    height=700
)

fig.show()

In [105]:
from statsmodels.stats.moment_helpers import cov2corr

y_approximated_all = []
coefs_all = []
errors_all = []
corr_matrixes_all = []
for i in range(len(models)):
    y_approximated_array = []
    coefs_array = []
    errors_array = []
    corr_matrixes = []
    for k in range(diffs.shape[2]):
        y_approximated, coefs, pcov = approximated(
            energies[1:13], res_all[i][k], error_sigma_all[i][k])
        y_approximated_array.append(y_approximated)
        coefs_array.append(coefs)
        errors_array.append(np.sqrt(abs(pcov.diagonal())))
        corr_matrixes.append(cov2corr(pcov))
        
    y_approximated_all.append(y_approximated_array)
    coefs_all.append(coefs_array)
    errors_all.append(errors_array)
    corr_matrixes_all.append(corr_matrixes)

corr_matrixes_all = np.array(corr_matrixes_all)

In [108]:
a = np.array(coefs_all).T[0]
b = np.array(coefs_all).T[1]
c = np.array(coefs_all).T[2]

In [109]:
reindex = [0, 3, 1, 4, 2, 5]
coefs_df_real = pd.DataFrame(
    np.hstack((coefs_all[1], errors_all[1]))
)[reindex]
coefs_df_real.columns = ['a', r'$\Delta a$', 'b', r'$\Delta b$', 'c', r'$\Delta c$']

coefs_df_real = coefs_df_real.round(3)
coefs_df_real.index = Rs
coefs_df_real[r'$\tilde{a}$'] = (coefs_df_real['a']**2 + (1.6 * coefs_df_real['b'])**2 + (coefs_df_real['c'] / 1.6)**2)**0.5 
coefs_df_real[r'$\Delta \tilde{a}$'] = np.sqrt( (coefs_df_real['a']*coefs_df_real[r'$\Delta a$'])**2 + \
                                           (2.56*coefs_df_real['b']*coefs_df_real[r'$\Delta b$'])**2 + \
                                           (coefs_df_real['c']*coefs_df_real[r'$\Delta c$'] / 2.56)**2) / coefs_df_real[r'$\tilde{a}$']

coefs_df_real[r'$\Delta \tilde{a}$'] = np.sqrt(
    coefs_df_real[r'$\Delta \tilde{a}$']**2 + 2 * (
        1.6**2 * coefs_df_real['a'] * coefs_df_real['b'] / coefs_df_real[r'$\tilde{a}$']**2 *\
        corr_matrixes_all[1, :, 0, 1] * coefs_df_real[r'$\Delta a$'] * coefs_df_real[r'$\Delta b$'] +\
        
        coefs_df_real['a'] * coefs_df_real['c'] / (coefs_df_real[r'$\tilde{a}$']**2 * 1.6**2) *\
        corr_matrixes_all[1, :, 0, 2] * coefs_df_real[r'$\Delta a$'] * coefs_df_real[r'$\Delta c$'] +\

        coefs_df_real['b'] * coefs_df_real['c'] / coefs_df_real[r'$\tilde{a}$']**2 *\
        corr_matrixes_all[1, :, 1, 2] * coefs_df_real[r'$\Delta b$'] * coefs_df_real[r'$\Delta c$']
    )
)

In [110]:
f_df = pd.read_csv('plots/f_df.csv')
f_df.index = Rs
coefs_df_real['f'] = f_df['f']
coefs_df_real['df'] = f_df['df']

In [112]:
df = coefs_df_real[['a', r'$\Delta a$', 'b', r'$\Delta b$', 'c', r'$\Delta c$', 'f', 'df']]
df.columns = ['a', 'da', 'b', 'db', 'c', 'dc', 'f', 'df']
df.to_csv('plots/res_params_fv_NN.csv', index=False)
df.to_csv('plots/res_params_fv_NN.tsv', sep='\t', index=False)

In [118]:
color = 'rgba(150, 0, 0, 0.3)'

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=Rs,
        y=coefs_df_real[r'$\tilde{a}$']+coefs_df_real[r'$\Delta \tilde{a}$'],
        line=dict(color='black', width=0.75),
    )
)

fig.add_trace(
    go.Scatter(
        x=Rs,
        y=coefs_df_real[r'$\tilde{a}$']-coefs_df_real[r'$\Delta \tilde{a}$'],
        fill='tonexty',
        fillcolor=color,
        line=dict(color='black', width=0.75)
    )
)

fig.add_trace(
    go.Scatter(
        x=Rs,
        y=coefs_df_real[r'$\tilde{a}$'],
        mode='lines+markers',
        line=dict(
            dash='dash',
            color='black'
        ),
        marker=dict(
            color='black',
            size=8,
            symbol='x'
        ),
    )
)

xaxis = lambda range: dict(
    showline=True,
    ticks='outside',
    mirror=True,
    tick0=1,
    dtick=0.2,
    range=range,
    linecolor='black',
    showgrid=True,
    gridcolor='grey',
    gridwidth=0.25,
)

yaxis = lambda range: dict(
    showline=True,
    ticks='outside',
    mirror=True,
    linecolor='black',
    range=range,
    showgrid=True,
    gridcolor='grey',
    gridwidth=0.25,
    zeroline=True,
    zerolinecolor='black',
    zerolinewidth=0.25
)

fig.update_layout(
    xaxis_title="R, m",
    yaxis_title=r'$\tilde{a}$',
    xaxis = xaxis([14, 17.2]),
    yaxis = yaxis([2.73, 2.855]),

    showlegend=False,
    height=500,
    width=950,
    font=dict(
            family="Times New Roman",
            size=16,
    ),
)

fig.show()
pio.write_image(fig, 'plots/atilde_vs_R_NN.pdf', height=500, width=950)